In [1]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy
a = numpy.random.randn(4,4)

In [2]:
a = a.astype(numpy.float32)

In [3]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [4]:
cuda.memcpy_htod(a_gpu, a)

In [5]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [6]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [7]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print a_doubled
print a

[[-0.99574608 -0.98367763  2.02555299  0.73084253]
 [-1.8184247  -1.0598712  -0.33329606  2.74019265]
 [-1.83399189  1.59599376 -0.79759067  3.51317167]
 [-1.75751126  1.40913594  2.62537527  1.83526289]]
[[-0.49787304 -0.49183881  1.01277649  0.36542127]
 [-0.90921235 -0.5299356  -0.16664803  1.37009633]
 [-0.91699594  0.79799688 -0.39879534  1.75658584]
 [-0.87875563  0.70456797  1.31268764  0.91763145]]
